In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import average_precision_score

Using TensorFlow backend.


In [3]:
outfile = '%s/partial-labels/experiments/DF_ResNet50_VOC2007.npz' % os.environ['HOME']
learning_rate = 0.1
nb_epoch = 20
batch_size = 32

npzfile = np.load(outfile)

X_train = npzfile['X_train']
Y_train = npzfile['Y_train']

X_test = npzfile['X_test']
Y_test = npzfile['Y_test']

print("data \n X_train=", X_train.shape, "Y_train=", Y_train.shape, " X_test=", X_test.shape, "Y_train=", Y_test.shape)

data 
 X_train= (5011, 2048) Y_train= (5011, 20)  X_test= (4952, 2048) Y_train= (4952, 20)


## model

In [4]:
model = Sequential()
model.add(Dense(20, input_dim=2048, name='fc1', activation='sigmoid'))
model.summary()


sgd = SGD(learning_rate)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['binary_accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 20)                40980     
Total params: 40,980
Trainable params: 40,980
Non-trainable params: 0
_________________________________________________________________


## train

In [5]:
model.fit(X_train, Y_train,batch_size=batch_size, epochs=nb_epoch,verbose=1)

scores = model.evaluate(X_test, Y_test, verbose=0)
print("%s TEST: %.2f%%" % (model.metrics_names[0], scores[0]*100))
print("%s TEST: %.2f%%" % (model.metrics_names[1], scores[1]*100))


Epoch 1/20
5011/5011 [==============================] - 1s 295us/step - loss: 0.1653 - binary_accuracy: 0.9450
Epoch 2/20
5011/5011 [==============================] - 0s 63us/step - loss: 0.1042 - binary_accuracy: 0.9648
Epoch 3/20
5011/5011 [==============================] - 0s 63us/step - loss: 0.0910 - binary_accuracy: 0.9692
Epoch 4/20
5011/5011 [==============================] - 0s 63us/step - loss: 0.0838 - binary_accuracy: 0.9711
Epoch 5/20
5011/5011 [==============================] - 0s 63us/step - loss: 0.0790 - binary_accuracy: 0.9729
Epoch 6/20
5011/5011 [==============================] - 0s 62us/step - loss: 0.0754 - binary_accuracy: 0.9734
Epoch 7/20
5011/5011 [==============================] - 0s 62us/step - loss: 0.0724 - binary_accuracy: 0.9748
Epoch 8/20
5011/5011 [==============================] - 0s 62us/step - loss: 0.0698 - binary_accuracy: 0.9756
Epoch 9/20
5011/5011 [==============================] - 0s 63us/step - loss: 0.0677 - binary_accuracy: 0.9764
Epoch 10/

## eval

In [6]:
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
AP_train = np.zeros(20)
AP_test = np.zeros(20)

for c in range(20):
    AP_train[c] = average_precision_score(Y_train[:, c], y_pred_train[:, c])
    AP_test[c] = average_precision_score(Y_test[:, c], y_pred_test[:, c])

print("MAP TRAIN =%.2f", AP_train.mean()*100)
print("MAP TEST =%.2f", AP_test.mean()*100)


MAP TRAIN =%.2f 91.70608549586493
MAP TEST =%.2f 82.81931017630104
